<a href="https://colab.research.google.com/github/Evans-tats/CodeAlpha_Emotion-Recognition-from-Speech/blob/main/Emotion_Recognition_from_Speech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import numpy as np

import librosa
import librosa.display
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

import IPython.display as ipd
from IPython.display import Audio

import os

In [1]:
Tess = "/kaggle/input/toronto-emotional-speech-set-tess/tess toronto emotional speech set data/TESS Toronto emotional speech set data/"
Ravdess = "/kaggle/input/ravdess-emotional-speech-audio/audio_speech_actors_01-24/"

In [8]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"evanskipsang","key":"ff66d8f0e99e93411cd7356a4a979f33"}'}

In [9]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [10]:
!kaggle datasets download -d uwrfkaggler/ravdess-emotional-speech-audio


Dataset URL: https://www.kaggle.com/datasets/uwrfkaggler/ravdess-emotional-speech-audio
License(s): CC-BY-NC-SA-4.0
 98% 420M/429M [00:05<00:00, 69.8MB/s]
100% 429M/429M [00:06<00:00, 75.0MB/s]


In [ ]:
!unzip ravdess-emotional-speech-audio.zip -d ravdess_data


In [7]:
file_emotion = []
file_path = []
data = ravadess_data
for i in ravdess_data/:
    # as their are 24 different actors in our previous directory we need to extract files for each actor.
    actor = os.listdir(Ravdess + i)
    for f in actor:
        part = f.split('.')[0].split('-')
    # third part in each file represents the emotion associated to that file.
        file_emotion.append(int(part[2]))
        file_path.append(Ravdess + i + '/' + f)

print(actor[0])
print(part[0])
print(file_path[0])
print(int(part[2]))
print(f)

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/ravdess-emotional-speech-audio/audio_speech_actors_01-24/'